In [39]:
from Pyleste.PICO8 import PICO8
from Pyleste.Carts.Celeste import Celeste
import Pyleste.CelesteUtils as utils

In [40]:
p8 = PICO8(Celeste)
utils.load_room(p8, 0)
utils.skip_player_spawn(p8)
LEVEL_GOAL = (108, 0)

utils.place_maddy(p8, 0, 120)

#p8.set_btn_state(38)

# 10 = alive and dashing
# 50 = dead, RIP :(
for _ in range(10):
    p8.step()

print(p8.game)

██████████████████████████    ██
██████████████                ██
████████                      ██
██████                    ██████
██▓▓▓▓                  ████████
██▓▓▓▓                      ████
██████████                    ██
████                          ██
██                            ██
██                        ██████
                            ████
              ████          ████
      ████    ████    ʌʌʌʌʌʌ████
██████████    ████ʌʌʌʌ██████████
██████████ʌʌʌʌ██████████████████
:D██████████████████████████████



In [41]:
# get reduced state from emulator
def get_state(p8):
    state_obj = {
        'x':0,
        'y':0,
        'rem_x':0,
        'rem_y':0,
        'spd_x':0,
        'spd_y':0
    }

    p = p8.game.get_player()
    
    if not p:
        state_obj['x'] = p8.game.last_player_pos[0]
        state_obj['y'] = p8.game.last_player_pos[1]
        state_obj['rem_x'] = p8.game.last_player_rem[0]
        state_obj['rem_y'] = p8.game.last_player_rem[1]
        state_obj['spd_x'] = p8.game.last_player_rem[0]
        state_obj['spd_y'] = p8.game.last_player_rem[1]
        return state_obj
    
    state_obj['x'] = p.x
    state_obj['y'] = p.y
    state_obj['rem_x'] = p.rem.x
    state_obj['rem_y'] = p.rem.y
    state_obj['spd_x'] = p.spd.x
    state_obj['spd_y'] = p.spd.y

    return state_obj

get_state(p8)

{'x': 0, 'y': 120, 'rem_x': 0, 'rem_y': 0, 'spd_x': 0, 'spd_y': 0}

In [42]:
def reward(p8):
    p = p8.game.get_player()

    x, y, remx, remy, spdx, spdy, reward = 0, 0, 0, 0, 0, 0, 0
    if not p:
        x = p8.game.last_player_pos[0]
        y = p8.game.last_player_pos[1]
        remx = p8.game.last_player_rem[0]
        remy = p8.game.last_player_rem[1]
        spdx = p8.game.last_player_spd[0]
        spdy = p8.game.last_player_spd[1]
        # punish death
        reward -= 100
    else:
        x = p.x
        y = p.y
        remx = p.rem.x
        remy = p.rem.y
        spdx = p.spd.x
        spdy = p.spd.y

    # for this level, the bigger the x, the better
    reward += x
    # the lower the y, the better
    reward += 128 - y

    # the exit requires fast upper movement, so let's reward that a little
    # speed values are generally very controller, so I actually need
    # to give it a little bump so it makes a difference
    reward += 2 * spdx # bigger x = better
    reward += (-1) * 5 * spdy # -1 because smaller y = better

    return reward

reward(p8)

8

In [43]:
def render(p8):
    print(p8.game)

render(p8)

██████████████████████████    ██
██████████████                ██
████████                      ██
██████                    ██████
██▓▓▓▓                  ████████
██▓▓▓▓                      ████
██████████                    ██
████                          ██
██                            ██
██                        ██████
                            ████
              ████          ████
      ████    ████    ʌʌʌʌʌʌ████
██████████    ████ʌʌʌʌ██████████
██████████ʌʌʌʌ██████████████████
:D██████████████████████████████



In [44]:
def get_all_actions():
    return [
        0,   # no input
        1,   # l
        2,   # r
        16,  # z
        17,  # l + z
        18,  # r + z
        32,  # x
        33,  # l + x
        34,  # r + x
        36,  # u + x
        37,  # u + l + x
        38,  # u + r + x
        40,  # d + x
        41,  # d + l + x
        42   # d + r + x
    ]
get_all_actions()

[0, 1, 2, 16, 17, 18, 32, 33, 34, 36, 37, 38, 40, 41, 42]

In [ ]:
# auxiliary functions for action analysis
def compute_displacement(player):
    sign = lambda x: 1 if x > 0 else -1 if x < 0 else 0
    dx, dy = round(player.rem.x + player.spd.x), round(player.rem.y + player.spd.y)
    dx, dy = dx + sign(dx), dy + sign(dy)
    while player.is_solid(dx, 0): dx -= sign(player.spd.x)
    while player.is_solid(dx, dy): dy -= sign(player.spd.y)
    return dx, dy

def action_restrictions(game, player):
    dx, dy = compute_displacement(player)
    h_movement = abs(player.spd.x) <= 1
    can_jump = not player.p_jump and (player.grace - 1 > 0 or player.is_solid(-3 + dx, dy) or player.is_solid(3 + dx, dy) or player.is_solid(dx, 1 + dy))
    can_dash = player.djump > 0 or player.is_solid(dx, 1 + dy) or player.check(game.balloon, 0, 0) or player.check(game.fruit, 0, 0) or player.check(game.fly_fruit, 0, 0)
    return h_movement, can_jump, can_dash

def eval_actions(h_movement, can_jump, can_dash):
    ''' button states
        0b000000 -  0 - no input
        0b000001 -  1 - l
        0b000010 -  2 - r
        0b010000 - 16 - z
        0b010001 - 17 - l + z
        0b010010 - 18 - r + z
        0b100000 - 32 - x
        0b100001 - 33 - l + x
        0b100010 - 34 - r + x
        0b100100 - 36 - u + x
        0b100101 - 37 - u + l + x
        0b100110 - 38 - u + r + x
        0b101000 - 40 - d + x
        0b101001 - 41 - d + l + x
        0b101010 - 42 - d + r + x
    '''
    actions = [0b000000] if not h_movement else [0b000000, 0b000001, 0b000010]
    if can_jump:
        actions.extend([0b010000] if not h_movement else [0b010000, 0b010001, 0b010010])
    if can_dash:
        actions.extend([0b100000, 0b100001, 0b100010, 0b100100, 0b100101, 0b100110, 0b101000, 0b101001, 0b101010])
    return actions


def get_possible_actions(p8):
    p = p8.game.get_player()

    if not p: # dead player
        return []

    if p.dash_time != 0: return [0b000000]
    return eval_actions(*action_restrictions(p8.game, p))

#get_possible_actions(p8)

KeyboardInterrupt: 

In [ ]:
p8.game.frames * 30

240

In [ ]:
# euclidean distance
def distance_to_goal(p8):
    x_goal, y_goal = LEVEL_GOAL
    p = p8.game.get_player()
    if not p:
        x, y = p8.game.last_player_pos
    else:
        x, y = p.x, p.y
    return ((x - x_goal) ** 2 + (y - y_goal) ** 2) ** 0.5

distance_to_goal(p8)

16.1245154965971